# Query and/or Retrieve catalogs from the AL / JWST Server.

In [ ]:
# Example notebook to query and/or retrieve from the AL/JWST Server via its API.
#   Utilizes the ALWS_utils.ipynb notebook, which contains API helper classes and methods.
#
#  Written by: Tom Hicks. 5/13/2022.
#  Last Modified: Initial creation.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

%matplotlib inline

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.data import download_file
from astropy.visualization import MinMaxInterval, AsinhStretch, SqrtStretch, ImageNormalize
import astropy.units as u

In [ ]:
import pandas as pd
from io import StringIO

#### Load additional "helper" code to help manage interaction with the API:

In [ ]:
%run ALWS_utils.ipynb

#### Create an instance of the API "helper" class which uses the default server base URL (both just loaded).

In [ ]:
api = AlwsApi()

#### Authorize an existing user, storing the resulting JWT access token in the instance of the class

In [ ]:
## Uncomment the following two lines to see the response when authorization fails:
# resp = api.authorize('jfake@nowhere.com', 'invalid_pwd')
# print(resp.json())
resp = api.authorize('jfake@nowhere.com', 'testydjango2')
# print(resp.json())

## Fetch a catalog file (CSV) from the Astrolabe image server

### First, let's see what catalogs are available from the server:

In [ ]:
resp = api.get('cats/names/')
get_results(resp)

### Let's get more information about the 'small10' catalog:

In [ ]:
get_results(api.get('cats/by_name/', params={'catalog': 'small10'}))

### This catalog looks like a good test case. Fetch the entire catalog and read it into a Pandas DataFrame:
#### _NOTE:_ Use this call carefully, as your computer may not have enough resources to do this for large catalogs.

In [ ]:
resp = api.get('cats/fetch/', params={'catalog': 'small10'})
csv_string = StringIO(resp.text)
df = pd.read_csv(csv_string)
df

### Since this is a tiny catalog, we can also get its data as JSON:
#### _NOTE:_ Use this call carefully, as the JSON response takes more time and memory. (Not recommended for catalogs with more than ~2000 rows).

In [ ]:
resp = api.get('cats/rows/by_catalog/', params={'catalog': 'small10'})
jcat = get_results(resp)
jcat

#### Note that each JSON object, corresponding to a row, stores the original table values for that row in a dictionary called 'cols'. We can rearrange the rows into a list of dictionaries, making it easy to read them into a Pandas DataFrame:

In [ ]:
dfj = pd.DataFrame([rec.get('cols') for rec in jcat])
dfj

## Do a Cone Query for matching rows within a catalog:

#### First, get a cutout of 10 arc seconds from an image, with the specified filter, from the specified collection, which contains the specified point.